# LightGCN model RecSys

In [1]:
# import matplotlib.pyplot as plt
# import networkx as nx
# import numpy as np
# import pandas as pd
import yaml
# import glob
# import torch
# import torch.nn
from lightgcn import LightGCN
# from torch_geometric.nn import LightGCN
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
# from collections import defaultdict
from utils import *
pd.set_option('display.max_colwidth', None)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device # = "cpu"

device(type='cuda', index=0)

# Load Config

In [3]:
%cd ..
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)
config

/home/ying/GNN-eCommerce


{'base': {'random_seed': 42},
 'data': {'cosmetic_shop': 'data/raw/cosmetic-shop-ecommerce-events/',
  'preprocessed': 'data/preprocessed/'},
 'training': {'event_type_weights': {'view': 0.01,
   'cart': 0.1,
   'remove_from_cart': -0.09,
   'purchase': 1.0}},
 'reports': None}

# Load Interaction Matrix from csv

In [4]:
interaction_matrix = pd.read_csv(config['data']['preprocessed'] + "interaction_matrix.csv")
interaction_matrix = interaction_matrix.rename(columns={"product_id": "item_id"})

In [5]:
im = interaction_matrix[['user_id', 'item_id', 'weight']].copy()

In [6]:
print('Total data size: ', len(im), ', unique user: ', im.user_id.nunique(), ', unique items: ', im.item_id.nunique())

Total data size:  10157408 , unique user:  1639358 , unique items:  54571


In [7]:
# ?? Should user never purchase removed at the beginning ??? NO for now!!
# mini_im = purchase_users(im)
#mini_im = im.sample(200000, random_state=1)  #100000
mini_im = im

In [8]:
print('Mini dataset size: ', len(mini_im), ', Users at least purchased once: ', len(purchase_users(mini_im)))
# print('Valid data percentage: ', f'{len(mini_im)/len(im):.2%}')

Mini dataset size:  10157408 , Users at least purchased once:  4688157


### Prepare Train/ Val/ Test dataset

In [9]:
train_df, test_df = train_test_split(mini_im, test_size=0.02)
test_df, val_df = train_test_split(test_df, test_size=0.5)

In [10]:
print('Mini set unique user: ', mini_im.user_id.nunique(), ', unique items: ', mini_im.item_id.nunique())
print("Train Size  : ", len(train_df))
print("Val Size : ", len (val_df))
print("Test Size : ", len (test_df))

Mini set unique user:  1639358 , unique items:  54571
Train Size  :  9954259
Val Size :  101575
Test Size :  101574


In [11]:
n_users, n_items, train_df, train_pos_list_df, val_pos_list_df, test_pos_list_df = prepare_val_test(train_df, val_df, test_df)  # , val_u_i_matrix, test_u_i_matrix

In [12]:
print("Users : ", n_users)
print("Items : ", n_items)
print("Train Size  : ", len(train_df))
print("Val Size : ", len (val_pos_list_df))
print("Test Size : ", len (test_pos_list_df))

Users :  1619629
Items :  54500
Train Size  :  9954259
Val Size :  10413
Test Size :  10474


In [13]:
# train_df

In [14]:
# train_pos_list_df

In [15]:
# val_pos_list_df

In [16]:
# test_pos_list_df

### Instantiate model and train/val the model

In [17]:
latent_dim = 64
n_layers = 3
LR = 0.005

model = LightGCN(num_nodes=n_users+n_items, embedding_dim=latent_dim, num_layers=n_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
print("Size of Learnable Embedding : ", [x.shape for x in list(model.parameters())])

Size of Learnable Embedding :  [torch.Size([1674129, 64])]


In [18]:
bpr_loss, reg_loss, final_loss, recall, precision = \
    train_and_evl(n_users, n_items, 1, train_df, train_pos_list_df, val_pos_list_df, model, optimizer, device=device, EPOCHS = 50, BATCH_SIZE = 2048, K = 20, DECAY = 0.0001)  # val_u_i_matrix,

bpr_loss | reg_loss | final_loss | precision | recall


  0%|          | 0/50 [00:00<?, ?it/s]

0.4829 0.0001 0.4829 0.0026 0.0463
0.2428 0.0002 0.243 0.0029 0.0524
0.2042 0.0003 0.2045 0.0032 0.0581
0.1812 0.0004 0.1816 0.0034 0.0616
0.1631 0.0004 0.1634 0.0035 0.0645
0.1558 0.0004 0.1563 0.0036 0.0663
0.1454 0.0005 0.1459 0.0036 0.0666
0.1381 0.0005 0.1386 0.0037 0.0679
0.1361 0.0005 0.1366 0.0038 0.0688
0.1264 0.0005 0.127 0.0038 0.0692
0.1246 0.0006 0.1251 0.004 0.0725
0.1171 0.0006 0.1177 0.004 0.0732
0.1146 0.0006 0.1152 0.0041 0.0747
0.1111 0.0007 0.1117 0.0041 0.0751
0.1073 0.0007 0.1079 0.0041 0.0757
0.1047 0.0007 0.1054 0.0042 0.076
0.1032 0.0007 0.1039 0.0043 0.078
0.0972 0.0008 0.098 0.0043 0.0784
0.096 0.0008 0.0968 0.0042 0.0781
0.0954 0.0008 0.0962 0.0044 0.0799
0.0921 0.0008 0.0929 0.0044 0.08
0.0886 0.0009 0.0894 0.0044 0.0803
0.0859 0.0009 0.0867 0.0044 0.0808
0.0846 0.0009 0.0855 0.0044 0.0816
0.0827 0.0009 0.0836 0.0045 0.0833
0.0806 0.001 0.0815 0.0045 0.0835
0.0762 0.001 0.0772 0.0045 0.0827
0.0747 0.001 0.0757 0.0046 0.0837
0.0758 0.001 0.0769 0.0045 0.0836

In [ ]:
edge_index, edge_weight = df_to_graph(train_df, True)
edge_index

In [ ]:
idx = list(range(len(users)))
random.shuffle(idx)
loader = DataLoader(idx, batch_size=100, shuffle=True)
len(idx)

In [ ]:
model.train()
for batch in loader:
    optimizer.zero_grad()

    batch_usr = users[batch]
    batch_pos_items = pos_items[batch]
    batch_neg_items = neg_items[batch]

    batch_pos_neg_labels = batch_pos_neg_edges(batch_usr, batch_pos_items, batch_neg_items)

In [ ]:
batch_pos_neg_labels

In [ ]:
# batch_pos_neg_labels[1].min()

In [ ]:
out = model(edge_index, batch_pos_neg_labels, edge_weight)
out

In [ ]:
size = len(batch)

bpr_loss = model.recommendation_loss(out[:size], out[size:], 0) * size
reg_loss = regularization_loss(model.embedding.weight, size, batch_usr, batch_pos_items, batch_neg_items)
loss = bpr_loss + reg_loss

In [ ]:
# bpr_loss_batch_list = []
# reg_loss_batch_list = []
# final_loss_batch_list = []

In [ ]:
# bpr_loss_batch_list.append(bpr_loss.item())
# reg_loss_batch_list.append(reg_loss.item())
# final_loss_batch_list.append(loss.item())

In [ ]:
# bpr_loss = round(np.mean(bpr_loss_batch_list), 8)
# reg_loss = round(np.mean(reg_loss_batch_list), 8)
# final_loss = round(np.mean(final_loss_batch_list), 8)

In [ ]:
print("bpr loss: ", loss, "reg loss: ", reg_loss, "final loss", loss)

In [ ]:
# bpr_loss_epoch_list = []
# reg_loss_epoch_list = []
# final_loss_epoch_list = []
# recall_epoch_list = []
# precision_epoch_list = []

In [ ]:
model.eval()
with torch.no_grad():
    embeds = model.get_embedding(edge_index, edge_weight)   # ?? ???
    final_usr_embed, final_item_embed = torch.split(embeds, (n_users, n_items))

In [ ]:
# matrix = interact_matrix(train_df, n_users, n_items)

In [ ]:
test_topK_recall, test_topK_precision = get_metrics(final_usr_embed, final_item_embed, val_pos_list_df, 20)

In [ ]:
print('precision: ', test_topK_precision, 'recall: ', test_topK_recall)

In [ ]:
relevance_score = final_usr_embed @ final_item_embed.t()
relevance_score.shape

In [ ]:
topk_relevance_indices = torch.topk(relevance_score, 20).indices
topk_relevance_indices[0:6]

In [ ]:
topk_relevance_indices.shape

In [ ]:
# t = topk_relevance_indices
# t = torch.flatten(t)
# t.shape

In [ ]:
# torch.max(t)

In [ ]:
# torch.min(t)

In [ ]:
# aa = list([68,561,1949,2478,4236,5117,5320,6173])
# train_df.loc[train_df['user_id_idx'].isin(aa)]

In [ ]:
topk_relevance_indices_df = pd.DataFrame(topk_relevance_indices.cpu().numpy())
topk_relevance_indices_df

In [ ]:
topk_relevance_indices_df['top_rlvnt_itm'] = topk_relevance_indices_df.values.tolist()
topk_relevance_indices_df['top_rlvnt_itm']

In [ ]:
topk_relevance_indices_df

In [ ]:
topk_relevance_indices_df['user_ID'] = topk_relevance_indices_df.index
topk_relevance_indices_df

In [ ]:
topk_relevance_indices_df = topk_relevance_indices_df[['user_ID', 'top_rlvnt_itm']]
topk_relevance_indices_df

In [ ]:
# test_interacted_items = val_df.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
# test_interacted_items

In [ ]:
test_pos_list_df

In [ ]:
metrics_df = pd.merge(test_pos_list_df, topk_relevance_indices_df, how='left', left_on='user_id_idx', right_on='user_ID')
metrics_df

In [ ]:
metrics_df['intrsctn_itm'] = [list(set(a).intersection(b)) for a, b in
                                  zip(metrics_df.item_id_idx_list, metrics_df.top_rlvnt_itm)]
metrics_df